<a href="https://colab.research.google.com/github/bt86548/104_crawler/blob/master/pytorch5%266.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [218]:
# import os     
# import zipfile

# local_zip = '/content/drive/My Drive/kagglecatsanddogs_3367a.zip' #需要解壓縮的檔案
# zip_ref = zipfile.ZipFile(local_zip, 'r')
# zip_ref.extractall('/content/PetImages2') #解壓縮的路徑
# zip_ref.close()

In [219]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

REBUILD_DATA = True


In [220]:
class DogsVSCats():
    IMG_SIZE = 50 #把所有圖片尺寸都統一,可能會失真需要先測試一下圖片
    CATS = "/content/PetImages2/PetImages/Cat" 
    DOGS = "/content/PetImages2/PetImages/Dog"
    TESTING = "PetImages/Testing"
    LABELS = {CATS: 0, DOGS: 1} #貼標籤
    training_data = []

    REBUILD_DATA = True
    catcount = 0 #計數很重要 需要知道做完沒
    dogcount = 0


    def make_training_data(self):
        for label in self.LABELS: #用標籤去跑所有的圖像
            for f in tqdm(os.listdir(label)): #跑遍所有圖片,tqdm進度條
                if "jpg" in f:
                    try:
                        path = os.path.join(label, f) #label為/content/PetImages2/PetImages/Cat位置,f為每張照片,以此類推去讀取
                        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE) #把讀到的照片變成灰階
                        img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE)) #把每張照片都變50*50的大小
                        self.training_data.append([np.array(img), np.eye(2)[self.LABELS[label]]])  
                        #np.array(img)為該圖片的矩陣,np.eye(2)[self.LABELS[label]]]
                        # do something like print(np.eye(2)[1]), just makes one_hot  np.eye(2)為對角線為1的2X2矩陣
                        #print(np.eye(2)[self.LABELS[label]]) 

                        if label == self.CATS:
                            self.catcount += 1
                        elif label == self.DOGS:
                            self.dogcount += 1
                    except Exception as e:
                        pass
                        #print(label, f, str(e))

        np.random.shuffle(self.training_data) #重新洗牌讓他更準
        np.save("training_data.npy", self.training_data) #存成npy檔讓圖片比較好辨識
        print('Cats:',dogsvcats.catcount) #計算貓照片的數量
        print('Dogs:',dogsvcats.dogcount) #計算狗照片的數量

if REBUILD_DATA:
    dogsvcats = DogsVSCats()
    dogsvcats.make_training_data()

100%|██████████| 12501/12501 [00:15<00:00, 803.21it/s]


Cats: 12476
Dogs: 12470


In [221]:
training_data = np.load("training_data.npy", allow_pickle=True) #讀取npy檔
print(len(training_data))

24946


In [222]:
# import torch

# X = torch.Tensor([i[0] for i in training_data]).view(-1,50,50)
# X = X/255.0
# y = torch.Tensor([i[1] for i in training_data])

In [223]:
# import matplotlib.pyplot as plt

# plt.imshow(X[0], cmap="gray")

In [224]:
# print(y[0])

In [225]:
import torch #建立模型
import torch.nn as nn
import torch.nn.functional as F


In [226]:
class Net(nn.Module):
    def __init__(self):
        #卷積層
        super().__init__() # just run the init of parent class (nn.Module)
        self.conv1 = nn.Conv2d(1, 32,   5,padding=1) # input is 1 image, 32 output channels, 5x5 kernel / window
        self.conv2 = nn.Conv2d(32, 128,  5,padding=1) # input is 32, bc the first layer output 32. Then we say the output will be 64 channels, 5x5 conv
        self.conv3 = nn.Conv2d(128, 256, 5,padding=1)

        #flattening.
        x = torch.randn(50,50).view(-1,1,50,50)
        self._to_linear = None
        self.convs(x)

        #扁平層:把多維的輸入壓扁為一維輸出，常用在從卷積層到全連接層的過渡，無參數'''
        self.fc1 = nn.Linear(self._to_linear, 512) 
        self.fc2 = nn.Linear(512, 2) # 512 in, 2 out bc we're doing 2 classes (dog vs cat).

    def convs(self, x): #三層池化層
        # max pooling over 2x2
        #取最大值(Max)：包括一維(MaxPooling1D)、二維(MaxPooling2D)、三維(MaxPooling3D)，分別處理時序資料、2D 圖形及每一點含其他資訊的2D 圖形。
        #池化層
        print("initial",x.shape)
        print()
        x = self.conv1(x)
        print("conv1", x.shape)
        print()
        x = F.relu(x)
        print("relu1",x.shape)
        print()
        x = F.max_pool2d(x, (3,3)) #max_pool取最大值
        print("max_pool2d 1",x.shape)

        print()
        x = self.conv2(x)
        print("conv2", x.shape)
        print()
        x = F.relu(x)
        print("relu2",x.shape)
        print()
        x = F.max_pool2d(x, (3,3))
        print("max_pool2d 2",x.shape)
        print()
        x = self.conv3(x)
        print("conv3", x.shape)
        print()
        x = F.relu(x)        
        print("relu3",x.shape)
        print()
        x = F.max_pool2d(x, (2,2))       
        print("max_pool2d 3",x.shape)        
        print()
        # print(x)
        
        if self._to_linear is None:
            print(x[0].shape[0])
            print(x[0].shape[1])
            print(x[0].shape[2])
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        
        return x

        

    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)  # .view is reshape ... this flattens X before 
        x = F.relu(self.fc1(x))
        x = self.fc2(x) # bc this is our output layer. No activation here.
        return F.softmax(x, dim=1)


net = Net()
print(net)

initial torch.Size([1, 1, 50, 50])

conv1 torch.Size([1, 32, 48, 48])

relu1 torch.Size([1, 32, 48, 48])

max_pool2d 1 torch.Size([1, 32, 16, 16])

conv2 torch.Size([1, 128, 14, 14])

relu2 torch.Size([1, 128, 14, 14])

max_pool2d 2 torch.Size([1, 128, 4, 4])

conv3 torch.Size([1, 256, 2, 2])

relu3 torch.Size([1, 256, 2, 2])

max_pool2d 3 torch.Size([1, 256, 1, 1])

256
1
1
Net(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 128, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(128, 256, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=256, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)


In [228]:
import torch.optim as optim #設定優化器

optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_function = nn.MSELoss() 

In [229]:
X = torch.Tensor([i[0] for i in training_data]).view(-1,50,50)
X = X/255.0
y = torch.Tensor([i[1] for i in training_data])

In [230]:
VAL_PCT = 0.1  # lets reserve 10% of our data for validation
val_size = int(len(X)*VAL_PCT)
print(val_size)

2494


In [231]:
train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]

In [232]:
print(len(train_X), len(test_X))

22452 2494


In [233]:
BATCH_SIZE = 100
EPOCHS = 1

for epoch in range(EPOCHS):
    for i in tqdm(range(0, len(train_X), BATCH_SIZE)): # from 0, to the len of x, stepping BATCH_SIZE at a time. [:50] ..for now just to dev
        #print(f"{i}:{i+BATCH_SIZE}")
        batch_X = train_X[i:i+BATCH_SIZE].view(-1, 1, 50, 50)
        batch_y = train_y[i:i+BATCH_SIZE]

        net.zero_grad()

        outputs = net(batch_X)
        loss = loss_function(outputs, batch_y)
        loss.backward()
        optimizer.step()    # Does the update

    print(f"Epoch: {epoch}. Loss: {loss}")

  0%|          | 0/225 [00:00<?, ?it/s]

initial torch.Size([100, 1, 50, 50])

conv1 torch.Size([100, 32, 48, 48])

relu1 torch.Size([100, 32, 48, 48])

max_pool2d 1 torch.Size([100, 32, 16, 16])

conv2 torch.Size([100, 128, 14, 14])

relu2 torch.Size([100, 128, 14, 14])

max_pool2d 2 torch.Size([100, 128, 4, 4])

conv3 torch.Size([100, 256, 2, 2])

relu3 torch.Size([100, 256, 2, 2])

max_pool2d 3 torch.Size([100, 256, 1, 1])



  0%|          | 1/225 [00:00<01:49,  2.05it/s]

initial torch.Size([100, 1, 50, 50])

conv1 torch.Size([100, 32, 48, 48])

relu1 torch.Size([100, 32, 48, 48])

max_pool2d 1 torch.Size([100, 32, 16, 16])

conv2 torch.Size([100, 128, 14, 14])

relu2 torch.Size([100, 128, 14, 14])

max_pool2d 2 torch.Size([100, 128, 4, 4])

conv3 torch.Size([100, 256, 2, 2])

relu3 torch.Size([100, 256, 2, 2])

max_pool2d 3 torch.Size([100, 256, 1, 1])



  1%|          | 2/225 [00:00<01:48,  2.05it/s]

...

initial torch.Size([100, 1, 50, 50])

conv1 torch.Size([100, 32, 48, 48])

relu1 torch.Size([100, 32, 48, 48])

max_pool2d 1 torch.Size([100, 32, 16, 16])

conv2 torch.Size([100, 128, 14, 14])

relu2 torch.Size([100, 128, 14, 14])

max_pool2d 2 torch.Size([100, 128, 4, 4])

conv3 torch.Size([100, 256, 2, 2])

relu3 torch.Size([100, 256, 2, 2])

max_pool2d 3 torch.Size([100, 256, 1, 1])



100%|█████████▉| 224/225 [01:44<00:00,  2.15it/s]

initial torch.Size([52, 1, 50, 50])

conv1 torch.Size([52, 32, 48, 48])

relu1 torch.Size([52, 32, 48, 48])

max_pool2d 1 torch.Size([52, 32, 16, 16])

conv2 torch.Size([52, 128, 14, 14])

relu2 torch.Size([52, 128, 14, 14])

max_pool2d 2 torch.Size([52, 128, 4, 4])

conv3 torch.Size([52, 256, 2, 2])

relu3 torch.Size([52, 256, 2, 2])

max_pool2d 3 torch.Size([52, 256, 1, 1])



100%|██████████| 225/225 [01:44<00:00,  2.15it/s]

Epoch: 0. Loss: 0.19856224954128265


In [234]:
correct = 0
total = 0
with torch.no_grad():
    for i in tqdm(range(len(test_X))):
        real_class = torch.argmax(test_y[i])
        net_out = net(test_X[i].view(-1, 1, 50, 50))[0]  # returns a list, 
        predicted_class = torch.argmax(net_out)

        if predicted_class == real_class:
            correct += 1
        total += 1
print("Accuracy: ", round(correct/total, 3))

  1%|          | 15/2494 [00:00<00:17, 145.03it/s]

initial torch.Size([1, 1, 50, 50])

conv1 torch.Size([1, 32, 48, 48])

relu1 torch.Size([1, 32, 48, 48])

max_pool2d 1 torch.Size([1, 32, 16, 16])

conv2 torch.Size([1, 128, 14, 14])

relu2 torch.Size([1, 128, 14, 14])

max_pool2d 2 torch.Size([1, 128, 4, 4])

conv3 torch.Size([1, 256, 2, 2])

relu3 torch.Size([1, 256, 2, 2])

max_pool2d 3 torch.Size([1, 256, 1, 1])

initial torch.Size([1, 1, 50, 50])

conv1 torch.Size([1, 32, 48, 48])

relu1 torch.Size([1, 32, 48, 48])

max_pool2d 1 torch.Size([1, 32, 16, 16])

conv2 torch.Size([1, 128, 14, 14])

relu2 torch.Size([1, 128, 14, 14])

max_pool2d 2 torch.Size([1, 128, 4, 4])

conv3 torch.Size([1, 256, 2, 2])

relu3 torch.Size([1, 256, 2, 2])

max_pool2d 3 torch.Size([1, 256, 1, 1])

initial torch.Size([1, 1, 50, 50])

conv1 torch.Size([1, 32, 48, 48])

relu1 torch.Size([1, 32, 48, 48])

max_pool2d 1 torch.Size([1, 32, 16, 16])

conv2 torch.Size([1, 128, 14, 14])

relu2 torch.Size([1, 128, 14, 14])

max_pool2d 2 torch.Size([1, 128, 4, 4])


  2%|▏         | 44/2494 [00:00<00:17, 143.52it/s]

conv3 torch.Size([1, 256, 2, 2])

relu3 torch.Size([1, 256, 2, 2])

max_pool2d 3 torch.Size([1, 256, 1, 1])

initial torch.Size([1, 1, 50, 50])

conv1 torch.Size([1, 32, 48, 48])

relu1 torch.Size([1, 32, 48, 48])

max_pool2d 1 torch.Size([1, 32, 16, 16])

conv2 torch.Size([1, 128, 14, 14])

relu2 torch.Size([1, 128, 14, 14])

max_pool2d 2 torch.Size([1, 128, 4, 4])

conv3 torch.Size([1, 256, 2, 2])

relu3 torch.Size([1, 256, 2, 2])

max_pool2d 3 torch.Size([1, 256, 1, 1])

initial torch.Size([1, 1, 50, 50])

conv1 torch.Size([1, 32, 48, 48])

relu1 torch.Size([1, 32, 48, 48])

max_pool2d 1 torch.Size([1, 32, 16, 16])

conv2 torch.Size([1, 128, 14, 14])

relu2 torch.Size([1, 128, 14, 14])

max_pool2d 2 torch.Size([1, 128, 4, 4])

conv3 torch.Size([1, 256, 2, 2])

relu3 torch.Size([1, 256, 2, 2])

max_pool2d 3 torch.Size([1, 256, 1, 1])

initial torch.Size([1, 1, 50, 50])

conv1 torch.Size([1, 32, 48, 48])

relu1 torch.Size([1, 32, 48, 48])

max_pool2d 1 torch.Size([1, 32, 16, 16])

con

  3%|▎         | 73/2494 [00:00<00:16, 142.47it/s]

...

relu2 torch.Size([1, 128, 14, 14])

max_pool2d 2 torch.Size([1, 128, 4, 4])

conv3 torch.Size([1, 256, 2, 2])

relu3 torch.Size([1, 256, 2, 2])

max_pool2d 3 torch.Size([1, 256, 1, 1])

initial torch.Size([1, 1, 50, 50])

conv1 torch.Size([1, 32, 48, 48])

relu1 torch.Size([1, 32, 48, 48])

max_pool2d 1 torch.Size([1, 32, 16, 16])

conv2 torch.Size([1, 128, 14, 14])

relu2 torch.Size([1, 128, 14, 14])

max_pool2d 2 torch.Size([1, 128, 4, 4])

conv3 torch.Size([1, 256, 2, 2])

relu3 torch.Size([1, 256, 2, 2])

max_pool2d 3 torch.Size([1, 256, 1, 1])

initial torch.Size([1, 1, 50, 50])

conv1 torch.Size([1, 32, 48, 48])

relu1 torch.Size([1, 32, 48, 48])

max_pool2d 1 torch.Size([1, 32, 16, 16])

conv2 torch.Size([1, 128, 14, 14])

relu2 torch.Size([1, 128, 14, 14])

max_pool2d 2 torch.Size([1, 128, 4, 4])

conv3 torch.Size([1, 256, 2, 2])

relu3 torch.Size([1, 256, 2, 2])

max_pool2d 3 torch.Size([1, 256, 1, 1])

initial torch.Size([1, 1, 50, 50])

conv1 torch.Size([1, 32, 48, 48])

rel

 99%|█████████▉| 2467/2494 [00:17<00:00, 140.66it/s]

max_pool2d 2 torch.Size([1, 128, 4, 4])

conv3 torch.Size([1, 256, 2, 2])

relu3 torch.Size([1, 256, 2, 2])

max_pool2d 3 torch.Size([1, 256, 1, 1])

initial torch.Size([1, 1, 50, 50])

conv1 torch.Size([1, 32, 48, 48])

relu1 torch.Size([1, 32, 48, 48])

max_pool2d 1 torch.Size([1, 32, 16, 16])

conv2 torch.Size([1, 128, 14, 14])

relu2 torch.Size([1, 128, 14, 14])

max_pool2d 2 torch.Size([1, 128, 4, 4])

conv3 torch.Size([1, 256, 2, 2])

relu3 torch.Size([1, 256, 2, 2])

max_pool2d 3 torch.Size([1, 256, 1, 1])

initial torch.Size([1, 1, 50, 50])

conv1 torch.Size([1, 32, 48, 48])

relu1 torch.Size([1, 32, 48, 48])

max_pool2d 1 torch.Size([1, 32, 16, 16])

conv2 torch.Size([1, 128, 14, 14])

relu2 torch.Size([1, 128, 14, 14])

max_pool2d 2 torch.Size([1, 128, 4, 4])

conv3 torch.Size([1, 256, 2, 2])

relu3 torch.Size([1, 256, 2, 2])

max_pool2d 3 torch.Size([1, 256, 1, 1])

initial torch.Size([1, 1, 50, 50])

conv1 torch.Size([1, 32, 48, 48])

relu1 torch.Size([1, 32, 48, 48])

max_

100%|██████████| 2494/2494 [00:17<00:00, 142.50it/s]

conv2 torch.Size([1, 128, 14, 14])

relu2 torch.Size([1, 128, 14, 14])

max_pool2d 2 torch.Size([1, 128, 4, 4])

conv3 torch.Size([1, 256, 2, 2])

relu3 torch.Size([1, 256, 2, 2])

max_pool2d 3 torch.Size([1, 256, 1, 1])

initial torch.Size([1, 1, 50, 50])

conv1 torch.Size([1, 32, 48, 48])

relu1 torch.Size([1, 32, 48, 48])

max_pool2d 1 torch.Size([1, 32, 16, 16])

conv2 torch.Size([1, 128, 14, 14])

relu2 torch.Size([1, 128, 14, 14])

max_pool2d 2 torch.Size([1, 128, 4, 4])

conv3 torch.Size([1, 256, 2, 2])

relu3 torch.Size([1, 256, 2, 2])

max_pool2d 3 torch.Size([1, 256, 1, 1])

initial torch.Size([1, 1, 50, 50])

conv1 torch.Size([1, 32, 48, 48])

relu1 torch.Size([1, 32, 48, 48])

max_pool2d 1 torch.Size([1, 32, 16, 16])

conv2 torch.Size([1, 128, 14, 14])

relu2 torch.Size([1, 128, 14, 14])

max_pool2d 2 torch.Size([1, 128, 4, 4])

conv3 torch.Size([1, 256, 2, 2])

relu3 torch.Size([1, 256, 2, 2])

max_pool2d 3 torch.Size([1, 256, 1, 1])

initial torch.Size([1, 1, 50, 50])

co